In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
# имена папок. они же - имена классов.
labels=['karabin','hryvnia','stepler','trash']
#извлекаем списки файлов для каждой папки.
filelists={}
for i in labels:
    filelists[i]=os.listdir(i)

# Сейчас надо схлопнуть один большой дескриптор (nx32 для ORB) из всех изображений.

In [3]:
orb=cv2.ORB_create()
sift=cv2.SIFT_create()
brisk=cv2.BRISK_create()

estimator=orb
name_estimator="orb"

des_list=[]
des_dict={}
failed_files=[] #файлы, для которых des=None
for i in labels:
    for j in filelists[i]:
        pathname=f'{i}/{j}'
        img=cv2.imread(pathname,0)
        img=cv2.resize( img, (img.shape[0]//4,img.shape[1]//4) )
        #img=cv2.resize(img, (400,300))
        kp,des=estimator.detectAndCompute(img,None)
        if(des is None):
            failed_files.append(pathname)
            continue
        des=des.astype('float32')
        des_list.append(des)
        des_dict[pathname]=des
        print(pathname, des.shape)

karabin/IMG_20201111_220200.jpg (500, 32)
karabin/(69).jpg (500, 32)
karabin/(73).jpg (500, 32)
karabin/(60).jpg (500, 32)
karabin/(23).jpg (490, 32)
karabin/(67).jpg (500, 32)
karabin/(25).jpg (500, 32)
karabin/(71).jpg (500, 32)
karabin/(82).jpg (500, 32)
karabin/(38).jpg (500, 32)
karabin/(1).jpg (500, 32)
karabin/(79).jpg (500, 32)
karabin/(40).jpg (500, 32)
karabin/(4).jpg (500, 32)
karabin/(63).jpg (500, 32)
karabin/(64).jpg (500, 32)
karabin/(76).jpg (500, 32)
karabin/(39).jpg (500, 32)
karabin/(83).jpg (500, 32)
karabin/(95).jpg (500, 32)
karabin/(34).jpg (500, 32)
karabin/(29).jpg (27, 32)
karabin/(12).jpg (500, 32)
karabin/(17).jpg (500, 32)
karabin/(14).jpg (500, 32)
karabin/(48).jpg (500, 32)
karabin/(35).jpg (500, 32)
karabin/(45).jpg (500, 32)
karabin/(5).jpg (500, 32)
karabin/(74).jpg (500, 32)
karabin/IMG_20201111_220155.jpg (500, 32)
karabin/(18).jpg (500, 32)
karabin/(87).jpg (500, 32)
karabin/(51).jpg (500, 32)
karabin/(56).jpg (500, 32)
karabin/(19).jpg (500, 32)
ka

hryvnia/(32).jpg (403, 32)
hryvnia/photo_2020-11-02_19-33-57.jpg (437, 32)
hryvnia/(27).jpg (461, 32)
hryvnia/(84).jpg (455, 32)
hryvnia/(98).jpg (91, 32)
stepler/(69).jpg (500, 32)
stepler/(73).jpg (500, 32)
stepler/(60).jpg (497, 32)
stepler/(23).jpg (500, 32)
stepler/(67).jpg (500, 32)
stepler/(25).jpg (361, 32)
stepler/(71).jpg (500, 32)
stepler/(82).jpg (500, 32)
stepler/(38).jpg (232, 32)
stepler/(1).jpg (500, 32)
stepler/(79).jpg (500, 32)
stepler/(40).jpg (500, 32)
stepler/(4).jpg (329, 32)
stepler/(63).jpg (500, 32)
stepler/(64).jpg (500, 32)
stepler/(76).jpg (469, 32)
stepler/(39).jpg (500, 32)
stepler/KIMG1092.jpg (500, 32)
stepler/(83).jpg (500, 32)
stepler/KIMG1113.jpg (494, 32)
stepler/(95).jpg (500, 32)
stepler/KIMG1089.jpg (500, 32)
stepler/(34).jpg (466, 32)
stepler/(29).jpg (500, 32)
stepler/(12).jpg (500, 32)
stepler/KIMG1122.jpg (500, 32)
stepler/(17).jpg (500, 32)
stepler/(14).jpg (500, 32)
stepler/(48).jpg (500, 32)
stepler/(35).jpg (317, 32)
stepler/(45).jpg (500

In [4]:
np.vstack(des_list).shape

(218528, 32)

In [ ]:
#тяжелая ячейка, комп начнет гудеть
from sklearn.cluster import KMeans
X=np.vstack(des_list)
kmeans=KMeans(n_clusters=10*4, random_state=0).fit( X )
print(kmeans.labels_)

In [ ]:
def create_bag(labels_list):
    res=np.zeros((10*4,))
    for i in labels_list:
        res[i]+=1
    return res
#Как работает bag-of words? Допустим, уникальные элементы - 0,1,2,3,4.
#Тогда [0,2,0,2,4,4,4,3,3,0,1] мы посчитаем частоты каждого ->[3,1,2,2,3]. 
#То что получилось - это 5 фичей, на основе которых будем проводить классификацию.
list_of_dicts=[]
for i in labels:
    for j in filelists[i]:
        pathname=f'{i}/{j}'
        if(pathname in failed_files):
            continue
        des=des_dict[pathname]
        kmeans_labels=kmeans.predict(des)
        bag_of_words=create_bag(kmeans_labels)
        temp_dict={}
        temp_dict['pathname']=pathname
        for k in np.arange(10*4):
            temp_dict[f'feature_{k}']=bag_of_words[k]
        temp_dict['target']=i
        list_of_dicts.append(temp_dict)

df=pd.DataFrame(list_of_dicts)
df.head()
df.to_csv(f"{name_estimator}_dataset.csv",index=False)

In [ ]:
from joblib import dump, load
dump(kmeans, f'written_models/{name_estimator}_kmeans.joblib') 
#dump(estimator,f'written_models/{name_estimator}_descriptor.joblib')